Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

# Quick Start: Running OpenFold on AWS Batch

## Table of Contents
0. [Install Dependencies](#0.-Install-Dependencies)
1. [Create Target](#1.-Create-Target)
2. [Submit Sequence Alignment and Folding Jobs](#2.-Submit-Sequence-Alignment-and-Folding-Jobs) 
3. [Download and Visualize Results](#3.-Download-and-Visualize-Results)

## 0. Install Dependencies

In [ ]:
# Import required Python packages

import boto3
from datetime import datetime
from batchfold.batchfold_environment import BatchFoldEnvironment
from batchfold.openfold_job import OpenFoldJob
from batchfold.batchfold_target import BatchFoldTarget
from batchfold.jackhmmer_job import JackhmmerJob
import matplotlib.pyplot as plt
from nbhelpers import nbhelpers

# Create AWS clients
boto_session = boto3.session.Session()
batch_environment = BatchFoldEnvironment(boto_session=boto_session)

S3_BUCKET = batch_environment.default_bucket
print(f" S3 bucket name is {S3_BUCKET}")

## 1. Create Target

In [ ]:
target_id = "7VNA"
target = BatchFoldTarget(target_id=target_id, s3_bucket=S3_BUCKET)
target.add_sequence(
    seq_id=target_id,
    seq="GSIPHKENMFKSKHKLDFSLVSMDQRGKHILGYADAELVNMGGYDLVHYDDLAYVASAHQELLKTGASGMIAYRYQKKDGEWQWLQTSSRLVYKNSKPDFVICTHRQLMDEEGHDLLGKR",
    description="Chain A|Ahr homolog spineless|Drosophila melanogaster",
)

## 2. Submit Sequence Alignment and Folding Jobs

In [ ]:
jackhmmer_job_name = (
    target.target_id + "_JackhmmerJob_" + datetime.now().strftime("%Y%m%d%s")
)
jackhmmer_job = JackhmmerJob(
    job_name=jackhmmer_job_name,
    target_id=target.target_id,
    fasta_s3_uri=target.get_fasta_s3_uri(),
    output_s3_uri=target.get_msas_s3_uri(),
    boto_session=boto_session,
    cpu=16,
    memory=31,
    db_preset="reduced_dbs",
)

job_name = target.target_id + "_OpenFoldJob_" + datetime.now().strftime("%Y%m%d%s")
openfold_job = OpenFoldJob(
    job_name=job_name,
    boto_session=boto_session,
    target_id=target.target_id,
    fasta_s3_uri=target.get_fasta_s3_uri(),
    msa_s3_uri=target.get_msas_s3_uri(),
    output_s3_uri=target.get_predictions_s3_uri() + "/" + job_name,
    use_precomputed_msas=True,
    config_preset="model_1_ptm",
    openfold_checkpoint_path="openfold_params/finetuning_ptm_2.pt",
    save_outputs=True,
    cpu=4,
    memory=15,  # Why not 16? ECS needs about 1 GB for container services
    gpu=1,
)

jackhmmer_submission = batch_environment.submit_job(
    jackhmmer_job, job_queue_name="GravitonSpotJobQueue"
)
openfold_submission = batch_environment.submit_job(
    openfold_job, job_queue_name="G4dnJobQueue", depends_on=[jackhmmer_submission]
)

Check on job statuses

In [ ]:
# for job in [jackhmmer_job, openfold_job]:
for job in [openfold_job]:
    print(
        f"Job {job.describe_job()[0]['jobName']} is in status {job.describe_job()[0]['status']}"
    )

## 3. Download and Visualize Results

Once the jobs are finished, download and view the results

### Plot Alignment Data

In [ ]:
target = BatchFoldTarget(
    target_id=target_id, s3_bucket=S3_BUCKET, boto_session=boto_session
)

In [ ]:
target.download_msas(local_path="data")
plt = nbhelpers.msa_plot(target_id, f"data/{target_id}/msas/jackhmmer")
plt.show()

### Plot Predicted Structure

In [ ]:
last_job_name = target.get_last_job_name(job_type="OpenFold")
print(f"Displaying predicted structure for job {last_job_name}")
target.download_predictions(local_path="data", job=last_job_name)
nbhelpers.pdb_plot(
    pdb_path=f"data/{target_id}/predictions/{last_job_name}", show_sidechains=False
).show()